In [ ]:
#머신러닝 라이브러리 설치
!pip install -U scikit-learn

In [ ]:
#시각화 라이브러리 설치
!pip install  plotly

In [ ]:
#머신러닝 관련 라이브러리
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras.layers as L
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import plotly.graph_objects as go
import tensorflow.keras.optimizers.schedules as schedules#학습률 스케줄링 알고리즘

In [ ]:
#데이터 불러오기
epochs=100
rootPath="../input/plant-pathology-2020-fgvc7/"
images= "images/"
test= "test.csv"
train = "train.csv"
result = "sample_submission.csv"

submission = pd.read_csv(rootPath+result)
testData = pd.read_csv(rootPath+test)
trainData = pd.read_csv(rootPath+train)
plusData=pd.read_csv("/kaggle/input/reallabel/plus.csv")


In [ ]:
#데이터불러오는 함수
def originPath(name):
    return rootPath + 'images/' + name + '.jpg'
#추가 데이터불러오는 함수
def plusPath(name):
    return "../input/plusimg/" + name + '.jpg'

In [ ]:
#데이터불러오는코드
testPaths = testData.image_id.apply(originPath).values
trainPaths = trainData.image_id.apply(originPath).values
plusPaths = plusData.image_id.apply(plusPath).values
trainLabels = np.float32(trainData.loc[:, 'healthy':'scab'].values)
plusLabels = np.float32(plusData.loc[:, 'healthy':'scab'].values)
trainPaths, validPaths, trainLabels, validLabels =train_test_split(trainPaths, trainLabels, test_size=0.15)

In [ ]:
#원래데이터에 추가 데이터 합치기
trainLabels=np.concatenate((trainLabels,plusLabels))

In [ ]:
#원래데이터 경로에 추가경로 합치기
trainPaths=np.concatenate((trainPaths,plusPaths))

In [ ]:
#2048 1365 3 데이터의 shape

In [ ]:
1365//4#데이터가 큼으로 비율대로 자르기

In [ ]:
height=341
width=514

In [ ]:
#데이터 전처리 함수
def dataPreprocessing(filename, label=None, image_size=(width, height)):
    image = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.image.resize(image, image_size)
    if label is None:
        return image
    return image, label
    
#데이터 증가 함수
def dataAugment(image, label=None):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    if label is None:    
        return image
    return image, label

In [ ]:
#TPU설정
AUTO = tf.data.experimental.AUTOTUNE
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
strategy = tf.distribute.experimental.TPUStrategy(tpu)
#TPU상태에따른 바치사이즈 조정
batchSize = 16 * strategy.num_replicas_in_sync

In [ ]:
#이미지 불러옴
trainDataset = (
    tf.data.Dataset
    .from_tensor_slices((trainPaths, trainLabels))
    .map(dataPreprocessing, num_parallel_calls=AUTO)
    .map(dataAugment, num_parallel_calls=AUTO)
    .repeat()
    .shuffle(512)
    .batch(batchSize)
    .prefetch(AUTO)
)

validDataset = (
    tf.data.Dataset
    .from_tensor_slices((validPaths, validLabels))
    .map(dataPreprocessing, num_parallel_calls=AUTO)
    .batch(batchSize)
    .cache()
    .prefetch(AUTO)
)

testDataset = (
    tf.data.Dataset
    .from_tensor_slices(testPaths)
    .map(dataPreprocessing, num_parallel_calls=AUTO)
    .batch(batchSize)
)


In [ ]:
#9가지 모델에대한 여러가지 학습 스케줄 알고리즘과 옵티마이저 조합으로 위한 전략패턴사용 여기서 최소 9*4*4*30번 이상의 학습을 수행함 
from tensorflow.keras.models import clone_model
class Combination():
    
    def __init__(self):
        self.model=None
        self.optimizer=None
    
    def setModel(self,Model):
        model=Model
        self.model=model.getModel()
        print(self.model.summary())
    
    def setOptimizer(self,Optimizer):
        
        self.optimizer=Optimizer
        print(self.optimizer)
    
    
    def getModel(self):
        return self.model
        
    
    def onLearning(self,epochs=30):
        
        with strategy.scope(): 
            self.model.compile(optimizer=self.optimizer,
                          loss = 'categorical_crossentropy',
                          metrics=['categorical_accuracy'])
        
        stepPerEpoch = trainLabels.shape[0] // batchSize
        history = self.model.fit(
                    trainDataset,
                    epochs=epochs,
                    steps_per_epoch=stepPerEpoch,
                    validation_data=validDataset)
        return history

In [ ]:
def dataPreprocessing(filename, label=None, image_size=(512, 512)):
    image = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.image.resize(image, image_size)
    if label is None:
        return image
    return image, label
    

def dataAugment(image, label=None):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    if label is None:    
        return image
    return image, label

In [ ]:
#TPU설정 
AUTO = tf.data.experimental.AUTOTUNE
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
strategy = tf.distribute.experimental.TPUStrategy(tpu)
#TPU상태에따른 바치사이즈 조정
batchSize = 16 * strategy.num_replicas_in_sync

In [ ]:
#이미지 불러옴
trainDataset = (
    tf.data.Dataset
    .from_tensor_slices((trainPaths, trainLabels))
    .map(dataPreprocessing, num_parallel_calls=AUTO)
    .map(dataAugment, num_parallel_calls=AUTO)
    .repeat()
    .shuffle(512)
    .batch(batchSize)
    .prefetch(AUTO)
)

validDataset = (
    tf.data.Dataset
    .from_tensor_slices((validPaths, validLabels))
    .map(dataPreprocessing, num_parallel_calls=AUTO)
    .batch(batchSize)
    .cache()
    .prefetch(AUTO)
)

testDataset = (
    tf.data.Dataset
    .from_tensor_slices(testPaths)
    .map(dataPreprocessing, num_parallel_calls=AUTO)
    .batch(batchSize)
)


In [ ]:
#9가지 모델에대한 여러가지 학습 스케줄 알고리즘과 옵티마이저 조합으로 위한 전략패턴사용!! 여기서 최소 9*4*4*30번 이상의 학습을 수행함
from tensorflow.keras.models import clone_model
class Combination():
    
    def __init__(self):
        self.model=None
        self.optimizer=None
    
    def setModel(self,Model):
        model=Model
        self.model=model.getModel()
        print(self.model.summary())
    
    def setOptimizer(self,Optimizer):
        
        self.optimizer=Optimizer
        print(self.optimizer)
    
    
    def getModel(self):
        return self.model
        
    
    def onLearning(self,epochs=30):
        
        with strategy.scope(): 
            self.model.compile(optimizer=self.optimizer,
                          loss = 'categorical_crossentropy',
                          metrics=['categorical_accuracy'])
        
        stepPerEpoch = trainLabels.shape[0] // batchSize
        history = self.model.fit(
                    trainDataset,
                    epochs=epochs,
                    steps_per_epoch=stepPerEpoch,
                    validation_data=validDataset)
        return history

In [ ]:
class Model:
    def __init__(self):
        pass
    def getModel():
        pass

In [ ]:
class Optimizer:
    def __init__(self):
        pass
    def getOptimizer():
        pass

In [ ]:
#모델 불러오기
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.applications import ResNet152
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.applications import ResNet101V2
from tensorflow.keras.applications import ResNet152V2
#처음에는 디폴트로 512*512로 이미지 잘라서 사용, 모든 모델 구성을 동일하게 함(옵티마이저나 학습률 알고리즘을 고정시켯다는게 아님!!)

In [ ]:
class Vgg16(Model):
    def __init__(self):
        with strategy.scope():    
            self.Model=tf.keras.Sequential([VGG16(input_shape=(width, height, 3),
                                                      weights='imagenet',
                                                    include_top=False),
                                        L.GlobalAveragePooling2D(),
                                        L.Dense(trainLabels.shape[1],
                                                activation='softmax',kernel_initializer='he_normal')])#이모델에 relu가 사용됨으로 이 가중치 초기화도 시도했다고 강조
        
    def getModel(self):
        return self.Model
    

In [ ]:
class Vgg19(Model):
    def __init__(self):
        with strategy.scope():    
            self.Model=tf.keras.Sequential([VGG19(input_shape=(width, height, 3),
                                                    weights='imagenet',
                                                    include_top=False),
                                        L.GlobalAveragePooling2D(),
                                        L.Dropout(0.2),
                                        L.Dense(trainLabels.shape[1],
                                                activation='softmax')])
        
    def getModel(self):
        return self.Model
    

In [ ]:
class Inceptionv3(Model):
    def __init__(self):
        with strategy.scope():    
            self.Model=tf.keras.Sequential([InceptionV3(
                                                    weights="imagenet",
                                                    include_top=False),
                                        L.GlobalAveragePooling2D(),
                                        L.Dense(trainLabels.shape[1],
                                                activation='softmax')])
        
    def getModel(self):
        return self.Model
    

In [ ]:
class Resnet50(Model):
    def __init__(self):
        with strategy.scope():    
            self.Model=tf.keras.Sequential([ResNet50(input_shape=(width, height, 3),
                                                    weights='imagenet',
                                                    include_top=False),
                                        L.GlobalAveragePooling2D(),
                                        L.Dropout(0.2),
                                        L.Dense(trainLabels.shape[1],
                                                activation='softmax')])
        
    def getModel(self):
        return self.Model
    

In [ ]:
class Resnet101(Model):
    def __init__(self):
        with strategy.scope():    
            self.Model=tf.keras.Sequential([ResNet101(input_shape=(width, height, 3),
                                                    weights='imagenet',
                                                    include_top=False),
                                        L.GlobalAveragePooling2D(),
                                        L.Dropout(0.2),
                                        L.Dense(trainLabels.shape[1],
                                                activation='softmax')])
        
    def getModel(self):
        return self.Model
    

In [ ]:
class Resnet152(Model):
    def __init__(self):
        with strategy.scope():    
            self.Model=tf.keras.Sequential([ResNet152(input_shape=(width, height, 3),
                                                    weights='imagenet',
                                                    include_top=False),
                                        L.GlobalAveragePooling2D(),
                                        L.Dropout(0.2),
                                        L.Dense(trainLabels.shape[1],
                                                activation='softmax')])
        
    def getModel(self):
        return self.Model
    

In [ ]:
class Resnet50v2(Model):
    def __init__(self):
        with strategy.scope():    
            self.Model=tf.keras.Sequential([ResNet50V2(input_shape=(width, height, 3),
                                                    weights='imagenet',
                                                    include_top=False),
                                        L.GlobalAveragePooling2D(),
                                        L.Dropout(0.2),
                                        L.Dense(trainLabels.shape[1],
                                                activation='softmax')])
        
    def getModel(self):
        return self.Model
    

In [ ]:
class Resnet101v2(Model):
    def __init__(self):
        with strategy.scope():    
            self.Model=tf.keras.Sequential([ResNet101V2(input_shape=(width, height, 3),
                                                    weights='imagenet',
                                                    include_top=False),
                                        L.GlobalAveragePooling2D(),
                                        L.Dropout(0.2),
                                        L.Dense(trainLabels.shape[1],
                                                activation='softmax')])
        
    def getModel(self):
        return self.Model
    

In [ ]:
class Resnet152v2(Model):
    def __init__(self):
        with strategy.scope():    
            self.Model=tf.keras.Sequential([ResNet152V2(input_shape=(width, height, 3),
                                                    weights='imagenet',
                                                    include_top=False),
                                        L.GlobalAveragePooling2D(),
                                        L.Dropout(0.2),
                                        L.Dense(trainLabels.shape[1],
                                                activation='softmax')])
        
    def getModel(self):
        return self.Model
#여기서 Resnet v2 는 renet 업글버전임 참고 https://cdn-5f733ed3c1ac190fbc56ef88.closte.com/wp-content/uploads/2019/07/rv1_vs_rv2.png 요곳도 넣어줘

In [ ]:
class SGD(Optimizer):
    def setSchedules(self,schedules):    
        self.Optim=tf.keras.optimizers.SGD(momentum=0.9,learning_rate=schedules)
        return self.Optim

In [ ]:
class Adagrad(Optimizer):
    def setSchedules(self,schedules):    
        self.Optim=tf.keras.optimizers.Adagrad(epsilon=1e-6,learning_rate=schedules)
        return self.Optim

In [ ]:
class RMSprop(Optimizer):
    def setSchedules(self,schedules):    
        self.Optim=tf.keras.optimizers.RMSprop(rho=0.9, epsilon=1e-06,learning_rate=schedules)
        return self.Optim
    
        

In [ ]:
class Adam(Optimizer):
    def setSchedules(self,schedules):    
        self.Optim=tf.keras.optimizers.Adam(beta_1=0.9, beta_2=0.999,learning_rate=schedules)
        return self.Optim

In [ ]:
models={"Vgg16":Vgg16(),"Vgg19":Vgg19(),"Resnet50":Resnet50(),"Inceptionv3":Inceptionv3(),
        "Resnet101":Resnet101(),"Resnet152":Resnet152(),"Resnet50v2"\
        :Resnet50v2(),"Resnet101v2":Resnet101v2(),"Resnet152v2":Resnet152v2()}

In [ ]:
#이거 발표할때 까먹을거 같음 
#옵티마이저 역할이랑 하이퍼파라미터계수 의미 적어줘
optimizers={"SGD":SGD(),"Adagrad":Adagrad(),"RMSprop":RMSprop(),"Adam":Adam()}

In [ ]:
#이것도
learningRateSchedulers={"CosineDecay":schedules.CosineDecay(initial_learning_rate=0.001, decay_steps=1000, alpha=0.0)\
                       ,"CosineDecayRestarts":schedules.CosineDecayRestarts(initial_learning_rate=0.001, t_mul=2.0,m_mul=1.0,first_decay_steps=1000, alpha=0.001)\
                       ,"ExponentialDecay":schedules.ExponentialDecay(initial_learning_rate=0.01,decay_steps=50,decay_rate=0.96,staircase=True)\
                       ,"InverseTimeDecay":schedules.InverseTimeDecay(initial_learning_rate = 0.01,decay_steps = 1.0,decay_rate = 0.5)
                       }

In [ ]:
combination=Combination()

In [ ]:
#그래프함수
def display(training, validation, title,yTitle,epochs):     
        fig = go.Figure()

        fig.add_trace(go.Scatter(x=np.arange(1, epochs+1), mode='lines+markers', y=training, marker=dict(color="#dc143c"),name="Train"))

        fig.add_trace(
            go.Scatter(x=np.arange(1, epochs+1), mode='lines+markers', y=validation, marker=dict(color="#0080ff"),
                   name="Validation"))

        fig.update_layout(title_text=title, yaxis_title=yTitle, xaxis_title="Epochs", template="plotly_dark")
        fig.show()

In [ ]:
#학습 함수
def performanceCheck():
    for modelName,model in models.items():
        weights=model.getModel().get_weights()#<- 가중치가 이어서 학습하면 안됨 가중치 초기화
        for optimizerName,optimizer in optimizers.items():
            for learningRateSchedulerName,learningRateScheduler in learningRateSchedulers.items():
                model.getModel().set_weights(weights)
                combination.setModel(model)
                combination.setOptimizer(optimizer.setSchedules(learningRateScheduler))
                history=combination.onLearning(epochs=30)
                trainAcc=history.history['categorical_accuracy']
                evalAcc=history.history['val_categorical_accuracy']
                trainLoss=history.history['loss']
                evalLoss=history.history['val_loss']
                title=modelName+" "+optimizerName+" "+learningRateSchedulerName+" "
                display(trainAcc,evalAcc,title+"Accuracy","Accuracy")
                display(trainLoss,evalLoss,title+"Loss","Loss")